In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from tf_keras.layers import Dense
from tf_keras.models import Sequential
from tf_keras.callbacks import EarlyStopping

In [ ]:
# inputs
#########
NAME = 'jaylen_brown'
model = LinearRegression()
#########

train = pd.read_csv(f"./player_game_logs/{NAME}/{NAME}_TRAIN.csv")
test = pd.read_csv(f"./player_game_logs/{NAME}/{NAME}_TEST.csv")

X_train = train.drop(columns=['PTS'])
X_test = test.drop(columns=['PTS'])
y_train = train['PTS']
y_test = test['PTS']
n_train = y_train.count()
n_test = y_test.count()
test_mean = np.mean(y_test)

# scale features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model.fit(X_train_scaled, y_train)

y_train_pred = model.predict(X_train_scaled)
mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, y_train_pred)

y_pred_test = model.predict(X_test_scaled)
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, y_pred_test)

rounded_preds = []
for p in y_pred_test:
    rounded_preds.append(round(p, 4))

y = list(zip(y_test, rounded_preds))

print(f"player: {NAME}\n")
print(f"(actual_pts, predicted_pts) pairs: \n{str(y)}\n")
print({f"{col}_weight": coef for col, coef in zip(X_train.columns, model.coef_)})
print(f"bias: {model.intercept_}")
print(f"y_test_mean: {test_mean}")
print(f"r2_train: {r2_train}")
print(f"r2_test: {r2_test}")

In [ ]:
## Feedforward NN playground
NAME = 'danny_manning'
train = pd.read_csv(f"./player_game_logs/{NAME}/{NAME}_TRAIN.csv")
test = pd.read_csv(f"./player_game_logs/{NAME}/{NAME}_TEST.csv")

X_train = train.drop(columns=['PTS'])
X_test = test.drop(columns=['PTS'])
y_train = train['PTS']
y_test = test['PTS']
n_train = y_train.count()
n_test = y_test.count()
test_mean = np.mean(y_test)

scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.to_numpy().reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.to_numpy().reshape(-1, 1)).flatten()

model = Sequential([
    Dense(87, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(20, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# train the model
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
history = model.fit(X_train_scaled, y_train_scaled,
epochs=20,
batch_size=32,
#validation_split=0.2,
callbacks=[early_stopping]
)

train_predictions = model.predict(X_train_scaled)
test_predictions = model.predict(X_test_scaled)

# scale back the predictions to the original value range
y_train_pred = scaler_y.inverse_transform(train_predictions.reshape(-1, 1)).flatten()
y_test_pred = scaler_y.inverse_transform(test_predictions.reshape(-1, 1)).flatten()

mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, y_test_pred)

rounded_preds = []
for p in y_test_pred:
    rounded_preds.append(round(p, 4))

y = list(zip(y_test, rounded_preds))

print(f"player: {NAME}\n")
print(f"(actual_pts, predicted_pts) pairs: \n{str(y)}\n")
print(f"rmse_train: {rmse_train}")
print(f"rmse_test: {rmse_test}")
print(f"r2_train: {r2_train}")
print(f"r2_test: {r2_test}")